<a href="https://colab.research.google.com/github/sightes/Scraping/blob/main/segundamano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import time
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
import re

#funcion Ligera
def check_inside(c_json,nombre,dev):
  if nombre in c_json:
        return(c_json[nombre] )
  else: 
        return(dev)

def check_inside_cadena(c_json,nombre,restante,dev):
  if check_inside(c_json,nombre,0)!=0:
        return(eval('c_json'+restante) )
  else: 
        return(dev)

def Fast_Segundamano(op,n):
  #extraccion spot locales comerciales 
  requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
  spots=[]
  for n_page in range(1,100):
    url='https://www.segundamano.mx/anuncios/mexico/'+ op +'-inmuebles/oficinas-locales?pagina='+str(n_page)
    headers={'authority': 'www.segundamano.mx',
    'path': '/anuncios/mexico/'+ op +'-inmuebles/oficinas-locales',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'}
    session=requests.Session()
    session.verify=False 
    r = session.get( url.replace(' ',''), headers=headers )
    session.close()
    root = lxml.html.fromstring(r.text)
    spots=spots+[[i['item']['name'], i['item']['url']] for i in json.loads(root.xpath("////*/script[@type = 'application/ld+json']/text()")[0])['itemListElement']]
    time.sleep(10)
  Tabla=pd.DataFrame(spots,columns=['nombre','url'])
  return(Tabla)

In [4]:
Tabla_renta=Fast_Segundamano('renta',100)

In [9]:
Tabla_renta

,nombre,url
0,Renta de oficina en Polanco cerca del parque,https://www.segundamano.mx/anuncios/ciudad-de-...
1,LA MEJOR UBICACIÓN PARA ESTABLECER TU EMPRESA.,https://www.segundamano.mx/anuncios/ciudad-de-...
2,La Mejor Opción Para Oficina en Granada / Anahuac,https://www.segundamano.mx/anuncios/ciudad-de-...
3,Renta oficina en palmas,https://www.segundamano.mx/anuncios/ciudad-de-...
4,OFICINA EN COLONIA DEL VALLE,https://www.segundamano.mx/anuncios/ciudad-de-...
...,...,...
2767,"Oficina en renta piso 1, Insurgentes Sur,",https://www.segundamano.mx/anuncios/ciudad-de-...
2768,RENTA DE OFICINA EN ROMA NORTE,https://www.segundamano.mx/anuncios/ciudad-de-...
2769,"OFICINA EN RENTA, CORPORATIVO LOMAS. BLVD MANU...",https://www.segundamano.mx/anuncios/ciudad-de-...
2770,"Traspaso Barberia, Av. Paseo de las Palmas,",https://www.segundamano.mx/anuncios/ciudad-de-...


In [5]:
Tabla_venta=Fast_Segundamano('venta',100)

In [8]:
Tabla_renta.to_csv('spot_renta.csv')

In [ ]:
def scan_Segundamano(Tabla):
    data2=[]
    for url in Tabla.url:
      requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
      headers={ 
      'authority': 'www.segundamano.mx',
      'path': '/anuncios/mexico/venta-inmuebles/oficinas-locales',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'
      }
      session=requests.Session()
      session.verify=False 
      r = session.get( url.replace(' ',''), headers=headers )
      session.close()
      root = lxml.html.fromstring(r.text)
      r1=root.xpath("//script[contains(.,'var AD =')]")[0].text
      #values = re.findall(r'var .*?=\s*(.*?);', r1, re.DOTALL | re.MULTILINE)
      #data=values[0]
      step_1 = r1.split('var AD =')
      step_2 = " ".join(step_1[1].split())
      step_3 = step_2.split('var SMMX_AD_VIEW ')
      step_4= step_3[0].replace('}, {','}xxx{').split('xxx')
      c_json=json.loads(step_4[0])
      precio=check_inside(c_json['prices'][0],'price_value','')
      image='https://images.segundamano.mx/api/v1/smmx/images/' + check_inside(c_json['thumbnail'],'path',0) + '?rule=webp_web_gallery_2x'
      estado=check_inside(c_json['locations'][0],'label','')
      municipio=check_inside(c_json['locations'][0]['locations'][0],'label','')
      if c_json['locations'][0]['locations'][0]['locations']!=None :
          sector=check_inside(c_json['locations'][0]['locations'][0]['locations'][0],'label','')
      else :  
          sector=check_inside(c_json['locations'][0]['locations'][0],'label','')
      detalle=check_inside(c_json,'body','')
      fec_ini=check_inside(c_json['list_time'],'label','')
      tipo=check_inside(c_json['type'],'label','')
      bath=check_inside_cadena(c_json['ad_details'],'bathrooms',"['bathrooms']['single']['label']",'')
      mail=check_inside_cadena(c_json['ad_details'],'contact_email',"['contact_email']['single']['label']",'')
      tipo2=check_inside(c_json['ad_details']['estate_type']['single'],'label','')
      estacionamiento=check_inside_cadena(c_json['ad_details'],'parking_lots',"['parking_lots']['single']['label']",'')
      area=check_inside_cadena(c_json['ad_details'],'size',"['size']['single']['label']",'')
      requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
      url='https://webapi.segundamano.mx/nga/api/v1/public/klfst/' + c_json['list_id'].split('/')[-1] + '/phone?lang=es'
      headers={ 
      'authority': 'webapi.segundamano.mx',
      'path': '/nga/api/v1/public/klfst/' + c_json['list_id'].split('/')[-1] + '/phone?lang=es',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'
      }
      session=requests.Session()
      session.verify=False 
      r = session.get( url.replace(' ',''), headers=headers )
      session.close()
      root = lxml.html.fromstring(r.text)
      if check_inside(json.loads(r.text),'phones',0)!=0:
        if len(json.loads(r.text)['phones'])!=0 :
          telefono=check_inside(json.loads(r.text)['phones'][0],'value',0)
        else :  
          telefono=check_inside(json.loads(r.text)['phones'],'value',0)
      else :
        telefono=''
      data2=data2+[[precio,image,estado,municipio,sector,detalle,fec_ini,tipo,bath,mail,tipo2,estacionamiento,area,telefono]]
      time.sleep(10)
    return(data2)


In [ ]:
c_n=86
c_dataframe=pd.DataFrame(data2 ,columns=['precio','image','estado','municipio','sector','detalle','fec_ini','tipo','bath','mail','tipo2','estacionamiento','area','telefono'])
c_dataframe.index=np.asarray(range(c_n,c_n+len(data2))).tolist()
c_Tabla=Tabla.join(c_dataframe)
c_Tabla=c_Tabla.loc[np.where((c_Tabla.precio>=0).tolist())[0].tolist()]

In [ ]:
c_Tabla.to_csv('SegundaMano 509-1014.csv')

In [ ]:
c_Tabla

,nombre,url,precio,image,estado,municipio,sector,detalle,fec_ini,tipo,bath,mail,tipo2,estacionamiento,area,telefono
86,Local en Venta en Santa Lucía,https://www.segundamano.mx/anuncios/campeche/c...,2100000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Campeche,Campeche,Santa Lucía,"Taller mecánico, con oficina agregada y baño. ...",05 Sep 22 11:10 CDT,Vendo,,,Oficinas/locales,,266 m²,9818163570
87,LOCAL EN VENTA EL PALMITO,https://www.segundamano.mx/anuncios/sinaloa/cu...,1950000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Sinaloa,Culiacán,Industrial El Palmito,"Local comercial con área refrigerada, área al ...",05 Sep 22 11:04 CDT,Vendo,,,Oficinas/locales,,60 m²,6672599450
88,LOCAL COMERCIAL EN VENTA,https://www.segundamano.mx/anuncios/ciudad-de-...,950000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Ciudad de México,Tlalpan,Tlalcoligia,Local comercial en venta cerca del mercado de ...,05 Sep 22 10:53 CDT,Vendo,1,,Oficinas/locales,0,33 m²,0
89,7 Locales comerciales en venta en la curva,https://www.segundamano.mx/anuncios/estado-de-...,7300000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Estado de México,Nicolás Romero,Francisco Sarabia 1a. Sección,7 Locales comerciales en venta en la curva \n ...,05 Sep 22 10:24 CDT,Vendo,2,,Oficinas/locales,,360 m²,5553674005
90,"OFICINAS EN VENTA - Montebello, Mérida, Yucatán.",https://www.segundamano.mx/anuncios/yucatan/me...,2880000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Yucatán,Mérida,Montebello (Fracc.),Orión Business Hub es el nuevo concepto de ofi...,05 Sep 22 10:15 CDT,Vendo,,,Oficinas/locales,,28 m²,9994008374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Local en Venta en Aldea Savia Tulum,https://www.segundamano.mx/anuncios/quintana-r...,2056000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Quintana Roo,Tulum,Ya Ax Tulum,Local en Venta en Aldea Savia Tulum\n\n\nAldea...,03 Sep 22 14:56 CDT,Vendo,0,,Oficinas/locales,0,,9981860535
173,Local en Venta en Santa Monica Monterrey,https://www.segundamano.mx/anuncios/nuevo-leon...,3350000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Nuevo León,Monterrey,Los Ángeles Bosque Residencial,Local en Venta en Santa Monica Monterrey\n\n\n...,03 Sep 22 14:55 CDT,Vendo,0,,Oficinas/locales,0,,9981860535
174,Local en Venta en Monte Real Tuxtla,https://www.segundamano.mx/anuncios/chiapas/tu...,1200000.0,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Monte Real (Fracc.),Local en Venta en Monte Real Tuxtla Local en V...,03 Sep 22 14:54 CDT,Vendo,0,,Oficinas/locales,0,,9981860535
175,Oficina en Venta en Kun Cancun,https://www.segundamano.mx/anuncios/quintana-r...,3602912.0,https://images.segundamano.mx/api/v1/smmx/imag...,Quintana Roo,Benito Juárez,Otra,Colonia: Cancún\n\nOficina en Venta en Kun Can...,03 Sep 22 14:53 CDT,Vendo,0,,Oficinas/locales,0,,9981860535


In [ ]:
['sitio','fec_scr','lat','lon','estado','municipio','url','tipo_spot','divisa','precio','modalidad','disponible','json']